In [95]:
import os
import re
import json

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout
from keras.layers import SpatialDropout1D, Conv1D, MaxPooling1D
from gensim.models import Word2Vec
from keras.utils import np_utils
from keras.preprocessing import sequence
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from keras.regularizers import l2
import seaborn as sns
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import model_from_json

pd.options.display.max_rows = 100
pd.set_option('display.max_columns', None)

import pickle

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
from keras.utils import to_categorical

from keras.models import load_model

from config_src import config


In [ ]:
w2v_path = os.path.join(config.data_dir, "word_embedding/ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec.gz")
jooble_data_path = os.path.join(config.by_jobs_csv_dir, "by_jobs.csv")


In [ ]:
%%time
vectorizer = KeyedVectors.load_word2vec_format(w2v_path, binary=False)

In [ ]:
dataset = pd.read_csv(jooble_data_path, sep="\t")

In [ ]:
max_len = 200

num_encoder_tokens = 300
latent_dim = 400

num_decoder_tokens = 2
max_decoder_seq_length = max_len

batch_size = 64
epochs = 40

In [ ]:
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = model_from_json(f.read())
    model.load_weights(model_weights_filename)
    return model

encoder_model = load_model('encoder_model.json', 'encoder_model_weights.h5')
decoder_model = load_model('decoder_model.json', 'decoder_model_weights.h5')

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        """sampled_char = reverse_target_char_index[sampled_token_index]"""
        decoded_sentence.append(sampled_token_index)

        # Exit condition: either hit max length
        # or find stop character.
        if (len(decoded_sentence) == max_decoder_seq_length) or \
        (len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
maxlen = 200
def get_sample_nonlabeled(row):

    text = row["text_lemmas_tags"]
    x_sample = get_training_sample(vectorizer, text)
    x_sample = sequence.pad_sequences([x_sample], maxlen=max_len, dtype='float', padding="post", truncating="post")
    x_sample = np.array(x_sample)
  
      #print("x shape =", x_sample.shape)
  
    return x_sample
  

In [ ]:
def get_training_sample(model, text):
    """
    Gets 1 sample of training data
    
    :param  model text:
    :return str text:
    """
    tagged_list = text.split(" ")

    vec_list = []
    
    #converting word2vec 
    for word in tagged_list:
        try:
            vec_list.append(model[word])
        except:
            #print("Word " + word + " isn't in vocab. Embeding as zeros")
            vec_list.append(np.zeros(300))
    return vec_list
  

In [ ]:
%%time
indexes_labeled = []
n_test = 10

for i, row in dataset.loc[: n_test].iterrows():
    x = get_sample_nonlabeled(dataset.loc[i])
    predict = decode_sequence(x)
  
    if np.sum(predict) > 0:
        indexes_labeled.append(i)
    
    if i % 100 == 0:
        print(i)
    
print()
print("Total:")
print(len(indexes_labeled))

In [ ]:
%%time
index = 9

x = get_sample_nonlabeled(dataset.loc[index])

predict = decode_sequence(x)

print(np.sum(predict))

print(dataset.loc[index, "title"])
print()
print(dataset.loc[index, "text"])

In [ ]:
print(np.sum(predict))
words = dataset.loc[index, "text_lemmas"].split(" ")
words2 = dataset.loc[index, "text_normalized"].split(" ")
for i in range(0, 100):
    if len(words) > i:
        word = words[i]
        word2 = words2[i]
    else:
        word = "END"
    print(i, 'Predicted', predict[i], " - ", word, " - ", word2)

In [88]:
col = pd.DataFrame(columns=["requirement_normalized"])

In [89]:
col

,requirement_normalized


In [97]:
dataset_proc = dataset.loc[: 200]

In [98]:
%%time
n_test = 1000

for i, row in dataset_proc.loc[0: 200].iterrows():
    x = get_sample_nonlabeled(dataset_proc.loc[i])
    predict = decode_sequence(x)

        
    tokens = dataset_proc.loc[i, "text_normalized"].split(" ")
    req = []
    #assert(len(predict) > len(tokens))
    for j, token in enumerate(tokens):
        if j >= len(predict):
            break
        if predict[j] == 1:
            req.append(token)

    dataset_proc.loc[i, "requirement_normalized"] =  " ".join(req)
     
    
    if i % 100 == 0:
        dataset_proc.to_csv("dataset_proc.csv", sep='\t', header=True, index=False)
        print(i, "Saved")
    
print()
print("Total:")
print(len(indexes_labeled))

C:\My_programs\Miniconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\My_programs\Miniconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0 Saved
100 Saved
200 Saved

Total:
3
Wall time: 2min 37s


In [100]:
dataset_proc

,id,lang_text,lang_title,text,text_lemmas,text_lemmas_tags,text_normalized,text_tokens,title,title_lemmas,title_lemmas_tags,title_normalized,title_tokens,url,requirement_normalized
0,8253926506452218683,russian,russian,\r\r\r\n Предлагаем большой ассортимент растен...,предлагать большой ассортимент растение собств...,предлагать_VERB большой_ADJ ассортимент_NOUN р...,предлага больш ассортимент растен собствен про...,предлагаем большой ассортимент растений собств...,Саженцы декоративных и плодовых культур. Могил...,саженец декоративный плодовый культура могилев...,саженец_NOUN декоративный_ADJ плодовый_ADJ кул...,саженц декоративн плодов культур могил гарант,саженцы декоративных плодовых культур могилев ...,http://slanet.by/Sazhency_dekorativnyh_i_plodo...,
1,-6951509096940958272,russian,russian,Срочно отрисовать не сложный ч/б логотип в век...,срочно рисовать сложный логотип вектор,срочно_ADV рисовать_VERB сложный_ADJ логотип_N...,срочн отрисова сложн логотип вектор,срочно отрисовать сложный логотип вектор,Дизайнер логотипов. Срочно отрисовать не сложн...,дизайнер логотип срочно рисовать сложный логот...,дизайнер_NOUN логотип_NOUN срочно_ADV рисовать...,дизайнер логотип срочн отрисова сложн логотип ...,дизайнер логотипов срочно отрисовать сложный л...,https://www.fl.ru/projects/3085095/srochno-otr...,
2,8709047890904426760,russian,russian,курсовая работа по Экономика фирмы \r\r\nдисци...,курсовой работа экономика фирма дисциплина эко...,курсовой_ADJ работа_NOUN экономика_NOUN фирма_...,курсов работ экономик фирм дисциплин экономик ...,курсовая работа экономика фирмы дисциплина эко...,#СРОЧНО!курсовая работа по Экономика фирмы (уд...,срочно курсовой работа экономика фирма удаленно,срочно_ADV курсовой_ADJ работа_NOUN экономика_...,срочн курсов работ экономик фирм удален,срочно курсовая работа экономика фирмы удаленно,https://www.fl.ru/projects/3085048/srochnokurs...,
3,3844137534604474003,russian,russian,Воспользуюсь услугами помощника по хозяйству в...,воспользоваться услуга помощник хозяйство част...,воспользоваться_VERB услуга_NOUN помощник_NOUN...,воспольз услуг помощник хозяйств частн дом пре...,воспользуюсь услугами помощника хозяйству част...,Помощник по хозяйству.,помощник хозяйство,помощник_NOUN хозяйство_NOUN,помощник хозяйств,помощник хозяйству,https://www.kufar.by/Брест/Вакансии/Помощник_п...,
4,4924993067476806508,russian,russian,"И показать как делать, так как сайтов много.\r...",показывать делать сайт сайт vds скрипть,показывать_VERB делать_VERB сайт_NOUN сайт_NOU...,показа дела сайт сайт vds скрипт,показать делать сайтов сайт vds скрипт,Перенести сайт с на (удаленно),переносить сайт удаленно,переносить_VERB сайт_NOUN удаленно_ADV,перенест сайт удален,перенести сайт удаленно,https://www.fl.ru/projects/3085032/perenesti-s...,
5,-8546375017518166188,russian,russian,"Добрый вечер коллеги, дорогие фрилансеры.\r\r\...",добрый вечер коллега дорогой фрилансер сердечн...,добрый_ADJ вечер_NOUN коллега_NOUN дорогой_ADJ...,добр вечер коллег дорог фрилансер сердечн позд...,добрый вечер коллеги дорогие фрилансеры сердеч...,С наступающим Новым Годом! (удаленно),наступать новый год удаленно,наступать_VERB новый_ADJ год_NOUN удаленно_ADV,наступа нов год удален,наступающим новым годом удаленно,https://www.fl.ru/projects/3085092/s-nastupayu...,
6,-4677914238886049045,russian,russian,Только 31 декабря!\r\r\nВсего за 777 рублей!\r...,декабрь рубль контактный данные адрес телефон ...,декабрь_NOUN рубль_NOUN контактный_ADJ данные_...,декабр рубл контактн дан адрес телефон сайт em...,декабря рублей контактные данные адрес телефон...,База данных - Организации РФ! (удаленно),база данные организация рф удаленно,база_NOUN данные_NOUN организация_NOUN рф_PROP...,баз дан организац рф удален,база данных организации рф удаленно,https://www.fl.ru/projects/3085030/baza-dannyi...,
7,1454265527494451396,russian,russian,"Крупнейший образовательный центр ""Лидер"" пригл...",крупный образовательный центр лидер приглашать...,крупный_ADJ образовательный_

In [101]:
dataset_proc.to_csv("dataset_proc.csv", sep='\t', header=True, index=False)